In [49]:
"""Volume 2: Non-negative Matrix Factorization."""

import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt
import os
from imageio import imread
import warnings
warnings.filterwarnings("ignore")

from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error as mse

In [104]:
#Problems 1-2
class NMFRecommender:

    def __init__(self,random_state=15,rank=3,maxiter=200,tol=1e-3):
        """The parameter values for the algorithm"""
        # Define attributes
        self.random_state = random_state
        self.k =rank
        self.maxiter = maxiter
        self.tol = tol
        
    
    def initialize_matrices(self,m,n):
        """randomly initialize the W and H matrices,"""
        np.random.seed(self.random_state)   # Set random seed
        self.W = np.random.random((m, self.k))  # Store as attribute
        self.H = np.random.random((self.k, n))
        return self.W, self.H   # Return them

      
    def fit(self,V):
        """Fits W and H weight matrices using CVXPY"""
        # Set up nonnegative matrices of the correct shape
        m, n = np.shape(V)
        self.initialize_matrices(m,n)
        W = cp.Variable((m, self.k), nonneg=True)
        H = cp.Variable((self.k, n), nonneg=True)
    
        while True:
            # Define objective to solve for W
            obj = cp.Minimize(cp.norm(V - W @ self.H, "fro"))
            prob = cp.Problem(objective=obj)
            prob.solve()
            oldW = self.W
            self.W = W.value

            # Define objective to solve for H
            obj = cp.Minimize(cp.norm(V - self.W @ H, "fro"))
            prob = cp.Problem(objective=obj)
            prob.solve()
            oldH = self.H
            self.H = H.value

            # See if the norm is low enough
            if (np.linalg.norm(oldW - self.W, "fro") < self.tol and np.linalg.norm(oldH - self.H, "fro") < self.tol):
                break


    def reconstruct(self):
        """Reconstruct V matrix for comparison against the original V"""
        return self.W @ self.H   # Reoncstruct and return V


In [91]:
# Prob 1
NMF = NMFRecommender()
NMF.initialize_matrices(2, 3)
print(NMF.W)
print(NMF.H)

[[0.8488177  0.17889592 0.05436321]
 [0.36153845 0.27540093 0.53000022]]
[[0.30591892 0.30447436 0.11174128]
 [0.24989901 0.9176299  0.26414685]
 [0.71777369 0.86571503 0.80707948]]


In [103]:
V = np.array([[0, 1, 0, 1, 2, 2],
              [2, 3, 1, 1, 2, 2],
              [1, 1, 1, 0, 1, 1],
              [0, 2, 3, 4, 1, 1],
              [0, 0, 0, 0, 1, 0]])
NMF = NMFRecommender()
NMF.fit(V)
print("Reoncstructed V")
print(NMF.reconstruct())

Reoncstructed V
[[1.75030991e-03 9.78621675e-01 4.49053638e-02 9.74447623e-01
  2.10812272e+00 1.90227867e+00]
 [1.99642549e+00 2.84428317e+00 1.23253063e+00 8.71516107e-01
  2.06996039e+00 2.06398729e+00]
 [1.00708073e+00 1.30872590e+00 5.38955262e-01 2.60536350e-01
  8.61120509e-01 8.73319205e-01]
 [1.68443122e-04 2.00887148e+00 2.98656138e+00 4.00744150e+00
  9.94563078e-01 9.97921879e-01]
 [2.25461795e-08 2.04847641e-01 8.26878080e-10 1.97971779e-01
  4.52629742e-01 4.08077284e-01]]


In [115]:
def prob3():
    """Run NMF recommender on the grocery store example"""
    V = np.array(
        [
            [0, 1, 0, 1, 2, 2],
            [2, 3, 1, 1 ,2, 2],
            [1, 1, 1, 0, 1, 1],
            [0, 2, 3, 4, 1, 1],
            [0, 0, 0, 0, 1, 0]
        ]
    )
    NMF = NMFRecommender(rank=2)  # Create an instance
    NMF.fit(V)   # Fit it

    # Find the number of people with higher weights in compoment 2 than 1
    num = 0
    for i in range(len(NMF.H[0])):
        if(NMF.H[1, i] > NMF.H[0, i]): num += 1

    return NMF.W, NMF.H, num

W, H, num = prob3()
print("W")
print(W)
print("H")
print(H)
print()
print(f"{num} people have higher weights in component 2 than 1")

W
[[4.32357340e-01 1.55958368e+00]
 [7.28830824e-01 2.67353753e+00]
 [2.59497870e-01 1.23222299e+00]
 [2.77133682e+00 7.42173394e-08]
 [2.00262945e-09 3.68909407e-01]]
H
[[6.13701977e-08 7.26310744e-01 1.07593007e+00 1.44659281e+00
  3.52998322e-01 3.65555244e-01]
 [5.85630624e-01 7.72185164e-01 6.58447872e-02 7.09874825e-09
  8.03647818e-01 7.67521958e-01]]

4 people have higher weights in component 2 than 1


In [3]:

#get data
def get_faces(path="./faces94"):
    """Traverse the specified directory to obtain one image per subdirectory.
    Flatten and convert each image to grayscale.

    Parameters:
        path (str): The directory containing the dataset of images.

    Returns:
        ((mn,k) ndarray) An array containing one column vector per
            subdirectory. k is the number of people, and each original
            image is mxn.
    """
    # Traverse the directory and get one image per subdirectory.
    faces = []
    for (dirpath, dirnames, filenames) in os.walk(path):
        for fname in filenames:
            if fname[-3:]=="jpg":       # Only get jpg images.
                # Load the image, convert it to grayscale,
                # and flatten it into a vector.
                faces.append(np.ravel(imread(dirpath+"/"+fname, as_gray=True)))
                break
    # Put all the face vectors column-wise into a matrix.
    return np.transpose(faces)

def show(image, m=200, n=180, plt_show=False):
    """Plot the flattened grayscale 'image' of width 'w' and height 'h'.

    Parameters:
        image ((mn,) ndarray): A flattened image.
        m (int): The original number of rows in the image.
        n (int): The original number of columns in the image.
        plt_show (bool): if True, call plt.show() at the end
    """
    #scale image
    image = image / 255
    #reshape image
    image = np.reshape(image,(m,n))
    #show image
    plt.imshow(image,cmap = "gray")
    
    if plt_show:
        plt.show()


In [4]:
def prob4():
    """
        Gridsearch over rank, alpha and l1_ratio values to reconstruct 
        image using NMF. Plot all reconstructed images.
    """
    raise NotImplementedError("Problem 3 incomplete")


In [ ]:
def prob5():
    '''
        find the 10 basis faces with the largest coefficients 
        corresponding to the the second and twelfth face in the dataset. 
        Plot these basis faces along with the original image using 
        subplots
    '''
    raise NotImplementedError("Problem 4 incomplete")